# Church & Temple Classification - Training Notebook

Here we will attempt a variety of techniques for classifying churches and temples from 11 different countries.  We assume that rotation has already been corrected by a separate process.

We have a few tricks up our sleeve to handle this kind of data.

1. Customize sampling rates from each country proportional to sqrt(N).  For instance, Russia has 11x as many images as Armenia, but will be sampled approximately 3.5x the rate of Armenia.
2. Sample/class weighting to increase the training weights of classes with few samples (Armenia, Australia)
3. Augment augment augment.  See `load_and_preprocess_data.ipynb`.  Every time an image is loaded from disk, it goes through a randomized set of augmentations, including random cropping, shearing, zooming, channel shift, horizontal flipping.